# TODOs for Krishna
- make sure environment works
- make sure paths are appropriate for your datasets

In [ ]:
import subprocess

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm
from rdkit import Chem, SimDivFilters
from rdkit.Chem import DataStructs
from sklearn.metrics import auc, precision_recall_curve, roc_auc_score

# First collect the data we'll be working with and validation sets

In [ ]:
# lets use the 37K+PK data as our training data
df = pd.read_csv("../data/data_prep_for_ml/data_prep_for_ml_pk_37k_screen/TRAIN_03_19_2022.csv")

# we'll hold out a random set from here for testing
# 20% test set will be sufficient since it is random and class balanced
# so for testing, use '../data/TEST_03_19_2022.csv'

# we'll also test on all the validation molecules we have predicted on
# since their scaffolds are quite different- easy,med,hard,molport
# so for testing, use '../data/cleaned_easy_med_hard_val_sets_800k_10_03_2022.csv'
df

In [ ]:
# Set paths for inputs and outputs
out_dir = "../out/experiment_to_test_different_negative_datasets_seeded"
fig_dir = "../figure_panels/negative_dataset_example_seeded"
bash_dir = "experiment_to_test_diff_neg_datasets_train_models_seeded.sh"
models_dir = "models/experiment_with_diff_neg_datasets_seeded"

# Let's split into sets where we have different proportions of positive:negative data

In [ ]:
pos = sum(df["hit"])
print("num pos: ", pos)

num_neg = [pos / 2, pos, pos * 2, pos * 4, pos * 8, pos * 16, len(df) - pos]
num_neg = [int(x) for x in num_neg]
print("number of neg to test:", num_neg)

# preprocess some pos and neg dfs and fp lists for later processing
negative_options = df[df["hit"] == 0.0]
positive_options = df[df["hit"] == 1.0]

all_neg_fps = [Chem.RDKFingerprint(Chem.MolFromSmiles(smi)) for smi in list(negative_options["SMILES"])]
all_pos_fps = [Chem.RDKFingerprint(Chem.MolFromSmiles(smi)) for smi in list(positive_options["SMILES"])]

# preprocess sorted indices for second method of selection (most similar to positive)
tan_sims_from_all_neg_to_pos = [
    max(DataStructs.BulkTanimotoSimilarity(query_fp, all_pos_fps)) for query_fp in tqdm.tqdm(all_neg_fps)
]
sorted_indices_most_similar = np.argsort(
    [1.0 - x for x in tan_sims_from_all_neg_to_pos]
)  # subtract from 1 so we argsort in most similar to least similar

# loop with random seed so we have multiple datasets for each size
num_datasets_per_size = 3  # Specify how many datasets you want per size
# note that for method 2, you'll always get the same dataset - thats fine
for seed in range(num_datasets_per_size):

    # Set random seed for reproducibility
    np.random.seed(seed)

    for i in num_neg:
        print("testing negative of #: " + str(i))

        # build negative sets in different ways
        # First way: random!
        rand_neg = negative_options.sample(n=i, replace=False, random_state=seed)
        rand = pd.concat([rand_neg, positive_options]).sample(
            frac=1, replace=False, random_state=seed
        )  # add to pos and shuffle
        rand.to_csv(out_dir + f"random_seed_{seed}_N_{i}.csv", index=False)

        # Second way: most similar to positive
        # Here, seeds only change the order of compounds in the df
        set_of_sorted_similar_indices = sorted_indices_most_similar[0:i]
        similar_neg = negative_options.iloc[set_of_sorted_similar_indices, :]
        similar = pd.concat([similar_neg, positive_options]).sample(
            frac=1, replace=False, random_state=seed
        )  # add to pos and shuffle
        similar.to_csv(out_dir + "similar_seed_{seed}_N_{i}.csv", index=False)

        # Third way: most diverse - use  MaxMin algorithm
        # (Ashton, M. et. al., Quant. Struct.-Act. Relat., 21 (2002), 598-604).
        # http://rdkit.blogspot.com/2014/08/picking-diverse-compounds-from-large.html
        # takes a ton of time for larger Ns
        mmp = SimDivFilters.MaxMinPicker(seed=seed)
        diverse_indices = mmp.LazyBitVectorPick(all_neg_fps, len(all_neg_fps), i)
        diverse_neg = negative_options.iloc[diverse_indices, :]
        diverse = pd.concat([diverse_neg, positive_options]).sample(frac=1)  # add to pos and shuffle
        diverse.to_csv(out_dir + "diverse_seed_{seed}_N_{i}.csv", index=False)

        # Now, let's prove to ourselves that we did this right by plotting one of the small examples
        # Batch similarity calculation for faster computation

        def calculate_pairwise_tanimoto(fps_indices, fps_list):
            """
            Compute pairwise Tanimoto similarities for a selected subset of fingerprints.

            Parameters:
            fps_indices (iterable): Indices (from the original dataframe) of the fingerprints to compare.
            fps_list (list): List of RDKit fingerprint objects for all negative molecules.

            Returns:
            np.ndarray: 1D array of pairwise Tanimoto similarities (upper triangle, no diagonal).
            """
            # Convert dataframe indices to positional indices for alignment with fps_list
            positional_indices = [negative_options.index.get_loc(idx) for idx in fps_indices]
            selected_fps = [fps_list[idx] for idx in positional_indices]

            similarity_matrix = np.zeros((len(selected_fps), len(selected_fps)))

            for i in range(len(selected_fps)):
                similarity_matrix[i, i:] = DataStructs.BulkTanimotoSimilarity(selected_fps[i], selected_fps[i:])

            # Extract upper triangle of the matrix (excluding the diagonal)
            pairwise_similarities = similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)]
            return pairwise_similarities

        if i < 1000:  # plot to confirm selection is working as intended
            # For random negatives
            random_tanimoto_scores = calculate_pairwise_tanimoto(rand_neg.index, all_neg_fps)
            plt.hist(random_tanimoto_scores, bins=30, label="random", alpha=0.7)

            # For similar negatives
            similar_tanimoto_scores = calculate_pairwise_tanimoto(
                negative_options.iloc[set_of_sorted_similar_indices].index, all_neg_fps
            )
            plt.hist(similar_tanimoto_scores, bins=30, label="similar", alpha=0.7)

            # For diverse negatives
            diverse_tanimoto_scores = calculate_pairwise_tanimoto(
                negative_options.iloc[diverse_indices].index, all_neg_fps
            )
            plt.hist(diverse_tanimoto_scores, bins=30, label="diverse", alpha=0.7)

            # Show plot
            plt.legend()
            plt.title("Pairwise Tanimoto Similarities")
            plt.xlabel("Tanimoto Similarity")
            plt.ylabel("Frequency")
            plt.tight_layout()
            plt.show()

# Prep script to train models on these different datasets

In [ ]:
for seed in range(num_datasets_per_size):
    for i in num_neg:
        for j in ["random", "similar", "diverse"]:
            # Skip non-random methods for the largest dataset
            if i == len(df) - pos and j != "random":
                continue

            # Names for folders and paths
            clean_name = f"{j}_seed_{seed}_N_{i}"
            models_fold = f"{models_dir}/{clean_name}/"
            curr_data_path = f"{out_dir}/{clean_name}.csv"

            mk_folder_command = f"mkdir -p {models_fold}"  # Use -p to avoid errors if the folder exists
            train_command = (
                f"chemprop_train --dropout 0.1 --hidden_size 500 --ffn_num_layers 2 --depth 3 "
                f"--metric prc-auc --extra_metrics auc --save_dir {models_fold} --data_path {curr_data_path} "
                f"--dataset_type classification --features_generator rdkit_2d_normalized --no_features_scaling "
                f"--num_folds 5 --ensemble_size 2 --split_type scaffold_balanced --split_sizes 0.8 0.1 0.1 "
                f"--smiles_columns SMILES --target_columns hit --gpu 0"
            )

            # Write the commands to the bash script
            with open(bash_dir, "a") as file1:
                file1.write(mk_folder_command + "; " + train_command + "\n")

# Then, use script and train the models

# Use different neg dataset models to predict on 20% random test set + external val set

Note: The ../../chemprop/ path must exist and contain the predict.py script. We assume the relevant conda environment name is 'chemprop'.

In [ ]:
activate_command = "conda activate chemprop; "

# num_neg = [508, 1017, 2034, 4068, 8136, 16272, 29950]
for seed in range(num_datasets_per_size):
    for i in num_neg:
        for j in ["random", "similar", "diverse"]:
            for data_name, data in zip(
                ["external_val", "20%_random_test"],
                [
                    "cleaned_easy_med_hard_val_sets_800k_10_03_2022.csv",
                    "TEST_03_19_2022.csv",
                ],
            ):

                # Names for folders and paths
                clean_name = f"{j}_seed_{seed}_N_{i}"
                models_fold = f"{models_dir}/{clean_name}/"
                curr_data_path = f"{out_dir}/{clean_name}.csv"
                pred_path = f"{out_dir}/pred_{data_name}_{clean_name}.csv"

                try:
                    # Updated file path to reflect the new naming convention
                    testdf = pd.read_csv(pred_path)
                    continue  # already exists
                except FileNotFoundError:
                    print(clean_name, data)
                    run_command = (
                        f"python predict.py --test_path {out_dir}/{data} "
                        f"--checkpoint_dir {models_fold}/ --preds_path {pred_path} "
                        f"--features_generator rdkit_2d_normalized --no_features_scaling --smiles_column SMILES "
                        f"--ensemble_variance --gpu 0"
                    )
                    full_command = activate_command + run_command
                    test = subprocess.run(
                        full_command,
                        cwd="../../chemprop/",
                        shell=True,
                        capture_output=True,
                    )

# Quantify predictions

In [ ]:
def modeleval(y_true, y_pred, name=""):
    """
    Evaluates model predictions using AUROC and Precision-Recall AUC.

    Parameters:
    y_true (list or array-like): Ground-truth binary labels (0/1).
    y_pred (list or array-like): Predicted probabilities or scores (floats).
    name (str, optional): Optional model name or identifier for logging. Default is "".

    Returns:
    tuple: A tuple containing:
        - float: AUROC score.
        - float: Area under the Precision-Recall curve (AUPR).
    """
    # Compute auROC
    auroc = float(roc_auc_score(y_true, y_pred))

    # Compute Precision-Recall
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr = float(auc(recall, precision))

    return (auroc, pr)

In [ ]:
columns = ["Seed", "N (negatives)", "Data Selection", "Test Set", "Metric", "Value"]
results = pd.DataFrame(columns=columns)

for seed in range(num_datasets_per_size):
    for i in num_neg:
        for j in ["random", "similar", "diverse"]:
            for data_name, data in zip(
                ["external_val", "20%_random_test"],
                [
                    "cleaned_easy_med_hard_val_sets_800k_10_03_2022.csv",
                    "TEST_03_19_2022.csv",
                ],
            ):
                clean_name = f"{j}_seed_{seed}_N_{i}"

                true_path = f"{out_dir}/{data}"
                pred_path = f"{out_dir}/pred_{data_name}_{clean_name}.csv"

                try:
                    true = pd.read_csv(true_path)
                    true = list(true["hit"])

                    test = pd.read_csv(pred_path)
                    test = list(test["hit"])
                except FileNotFoundError:
                    continue

                # Calculate metrics
                roc, pr = modeleval(true, test)

                # Prepare rows for results
                row1 = [seed, i, j, data_name, "auROC", roc]
                row2 = [seed, i, j, data_name, "auPR", pr]
                new = pd.DataFrame([row1, row2], columns=columns)

                # Append results
                results = pd.concat([results, new], ignore_index=True)

In [ ]:
# rename for plotting
results["Test Set"] = [
    "Restricted Similarity" if x == "external_val" else "Random (20%)" for x in list(results["Test Set"])
]
results.to_csv(f"{fig_dir}_RESULTS_SEEDED.csv", index=False)
results

In [ ]:
results = results.reset_index(drop=True)
order = ["random", "similar", "diverse"]
palette = dict(zip(order, ["grey", "lightsteelblue", "royalblue"]))
sns.set(rc={"figure.dpi": 300, "savefig.dpi": 300})
sns.set_style("whitegrid", {"axes.grid": False})
g = sns.FacetGrid(
    results,
    row="Metric",
    col="Test Set",
    sharey=True,
    height=4,
    aspect=1.5,
    margin_titles=True,
)
g.map(
    sns.lineplot,
    "N (negatives)",
    "Value",
    "Data Selection",
    marker="o",
    dashes=True,
    hue_order=order,
    palette=palette,
)

for ax in g.axes_dict.values():
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
g.set(xscale="log")
g.set(xticks=num_neg)
g.set(xticklabels=num_neg)
plt.legend()
plt.tight_layout()
plt.savefig(fig_dir + ".png")
plt.savefig(fig_dir + ".svg")
plt.show()